In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import keras
import os
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers import Input, Conv2D, Dense, Activation, AveragePooling2D\
,Flatten, Lambda, Reshape, LocallyConnected2D, BatchNormalization, MaxPool2D, Dropout
from keras.models import Model
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
#Load data
dirname = '/kaggle/input/Kannada-MNIST'
output_dirname = '/kaggle/output/kaggle/working'

train_df = pd.read_csv(os.path.join(dirname, 'train.csv'))
test_df = pd.read_csv(os.path.join(dirname, 'test.csv'))
dig_df = pd.read_csv(os.path.join(dirname, 'Dig-MNIST.csv'))
sample_df = pd.read_csv(os.path.join(dirname, 'sample_submission.csv'))


In [ ]:
#Prepare data
X = train_df.drop('label', axis=1).to_numpy()
y = train_df['label'].to_numpy()
X_dig = dig_df.drop('label', axis=1).to_numpy()
y_dig = dig_df['label'].to_numpy()
X_test = test_df.drop('id', axis=1).to_numpy()

X = X.reshape(-1, 28, 28, 1)
y = to_categorical(y)
X_dig = X_dig.reshape(-1, 28, 28, 1)
y_dig = to_categorical(y_dig)
X_test = X_test.reshape(-1, 28, 28, 1)

X = X / 255
X_dig = X_dig / 255
X_test = X_test / 255

seed = 33
np.random.RandomState(seed)
valid_size=0.15

X_train, X_valid, y_train, y_valid = train_test_split(X, y,\
test_size=valid_size, random_state=seed)

<center>$\textbf{ЗАДАНИЕ 1}$<center>

In [ ]:
#Create Model
def create_model():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(filters=6, kernel_size=(5, 5), padding='Same', activation='tanh')(input_layer)
    x = AveragePooling2D()(x)
    x = Activation('tanh')(x)
    x = Conv2D(filters=16, kernel_size=(5, 5), activation='tanh')(x)
    x = AveragePooling2D()(x)
    x = Activation('tanh')(x)
    x = Conv2D(filters=120, kernel_size=(5, 5), activation='tanh')(x)
    x = Flatten()(x)
    x = Dense(84, activation='tanh')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=[input_layer], outputs=[output])
    
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
#model.fit(X_train, y_train, batch_size=16, epochs=30, verbose=1, validation_split=0.1)


Отмечу сразу,что в "LeNet-5" после пулинга я добавил как и гласила инструкция функцию активации "tanh", хотя я находил реализации где этой функции не было(и это логично, именно от этого псевдослоя пользы минимум). Точность на валидации 0.992. Вцелом, это признак того, что мы скорее хорошо обучились, чем переобучились, но если залить этот LeNet-5, то результат будет куда меньше, смею предположить, что это всё таки связано не с переобучением, а с тем, что выборка, которую нам дали не в полной мере представляет данные на которых будет произведенна проверка

<center>$\textbf{ЗАДАНИЕ 2}$<center>


In [ ]:
model = create_model()
model.summary()
model.compile(optimizer=keras.optimizers.SGD(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#model.fit(X_train, y_train, batch_size=512, epochs=30, verbose=1, validation_split=0.1,
#         callbacks=[ReduceLROnPlateau(factor=0.1, patieecne=1, min_delta=0.01),])


Увеличив размер батча, я тем самым уменьшил количество шагов градиентного спуска, что в свою очередь теоритический говорит о том, что модель недоучится. Что и подтверждается практикой, ибо точность на валидации 0.96. 

<center>$\textbf{ЗАДАНИЕ 3}$<center>

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        rotation_range=10, 
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False)


datagen.fit(X_train)

In [ ]:
#Create Model
def create_my_model():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu')(input_layer)
    x = Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu')(x)
    x = BatchNormalization(momentum=.15)(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu')(x)
    x = BatchNormalization(momentum=.15)(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu')(x)
    x = Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu')(x)
    x = BatchNormalization(momentum=.15)(x)
    x = MaxPool2D(pool_size=(2,2))(x)

    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.4)(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=[input_layer], outputs=[output])
    
    return model

In [ ]:
my_model = create_my_model()
my_model.summary()

In [ ]:
my_model.compile(optimizer=keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
my_model.fit(datagen.flow(X_train, y_train, batch_size=64), 
         epochs=30, 
         verbose=1, validation_data = (X_valid, y_valid),
         callbacks=[ReduceLROnPlateau(monitor='val_accuracy',
                                      factor=0.5, 
                                      patieecne=3,
                                      verbose=1,
                                      min_lr=0.00001),])

In [ ]:
y_result = my_model.predict(X_test)
y_result = np.argmax(y_result, axis=1)

sample_df['label'] = y_result
sample_df.to_csv('submission.csv', index=False)

<center>$\textbf{ЗАДАНИЕ 4}$<center>


На самом деле это не полностью мною придуманная нейросеть, а комбинация идей, уже
хорошо себя проявивших сетей на этом конкурсе.

1) Ну функция активации была использована 'relu', потому что 'tanh' быстро перенасыщяется

2) Ну после того как функция активации поменялась, то значения на выходе перестали 
быть номрализованными, поэтому да, конечно теперь полезно будет применять батч-нормализацию.

Ещё пару основных идей, что используются:

1) Достаточно полезным будет нагенеривание больше входных данных путём, модификации
данного обучающегося датасета. То есть небольшого поворота картинки или сдвига.

2) Применение большого свёрточного слоя, путём применения двух последоватеных 
маленьких свёрточных слоёв. Что уменьшит общее количество параметров. Именно
поэтому(я так думаю) не стоит между ними вставлять батчнормализации, ибо хоть
и используется "relu", но два этих слоя несут одну смысловую нагрузку.

3)Вместо AveragePooling используется MaxPool2D я так думаю, что это всё ради чёткого

выделения границы нарисованной фигуры,а не плавного перехода.

4)Dropout, что бы один нейрон не брал на себя весь функционал.




<center>$\textbf{ЗАДАНИЕ 5}$<center>


In [ ]:

optimizers = {
    'sgd': 'SGD',
    'sgd_mom': keras.optimizers.SGD(momentum=0.5),
    'rmsprop': 'RMSprop',
    'adam': 'Adam',
}


In [ ]:
historys = {}
models = {}
num_epochs = 1

for n, optimizer in enumerate(optimizers):
    models[optimizer] = create_my_model()
    
    models[optimizer].compile(optimizer=optimizers[optimizer],
             loss='categorical_crossentropy',
             metrics=['accuracy'])
    
    start = time.time()
    h = models[optimizer].fit(X_train, y_train, 
                              batch_size=1024, 
                              epochs=num_epochs, 
                              verbose=0, 
                              validation_data=(X_valid, y_valid),
                             callbacks=[ReduceLROnPlateau(monitor='val_accuracy',
                                      factor=0.5, 
                                      patieecne=3,
                                      verbose=1,
                                      min_lr=0.00001),
                                       CSVLogger("{0}.log".format(optimizer))]
                             )
    historys[optimizer] = h.history
    
    
    print("{0} Optimizer: ".format(optimizer))
    print("accuracy: {0}, val_accuracy {1}, time {2} minutes: ".format(\
        historys[optimizer]['accuracy'][-1], 
        historys[optimizer]['val_accuracy'][-1],
        (time.time() - start) / 60))                                             

In [ ]:
def plot_accuracy(historys, names, epochs=num_epochs, filter_n=1):
    plt.figure(figsize=(15, 5))
    styles=[':','-.','--','-']
    
    for n, h in enumerate(historys.values()):
        val_acc = h['val_accuracy']
            
        plt.plot(val_acc, linestyle=styles[n])
    
    plt.title('Model validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(names, loc='upper left')
    axes = plt.gca()
    axes.set_ylim([0.98, 0.997])
    axes.set_xlim([0, epochs-1])

In [ ]:
plot_accuracy(historys, optimizers.keys())

Именно смотрят на это график, я решил выбрать оптимизацию по "Adam". Такии скачки вниз у "rmsprop" скорее всего можно объяснить что мы попали в локальный минимум и
благополучно выбрались из него. Вряди ли это было связано с неудачным батчом, ибо размер батча был достаточно большим(1024). Ну наверно можно ещё сказать, что "sgd" с моментом быстрее обучается, связано это с тем, что он быстрее набирает скорость за счет инерции и локальыне минимумы его не стопят.